In [6]:
import pandas as pd
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

#### Pulled from class will need to add values

"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = [
'jtleek/datasharing',
'rdpeng/ProgrammingAssignment2',
'trimstray/the-book-of-secret-knowledge',
'ryanmcdermott/clean-code-javascript',
'gothinkster/realworld',
'vuejs/awesome-vue',
'tonsky/FiraCode',
'hakimel/reveal.js',
'angular/angular.js',
'reduxjs/redux',
'tailwindlabs/tailwindcss',
'ripienaar/free-for-dev',
'protocolbuffers/protobuf',
'shadowsocks/shadowsocks-windows',
'JetBrains/kotlin',
'yarnpkg/yarn',
'TryGhost/Ghost',
'square/retrofit',
'bradtraversy/design-resources-for-developers',
'vsouza/awesome-ios',
'iamkun/dayjs'
]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data():
    """
    This function checks for a local csv file of raw data. If a local csv exists it reads it 
    to a df.
    
    If no local csv of raw data exists, it loops through all of the repos in the REPOS list above and
    and runs the process functions to scrape the repo name, language, and readme
    contents. It then writes the resulting dictionary of lists to a df, returning
    a dataframe of raw data. It then caches the raw data to a local csv file.
    """
    if os.path.isfile('raw_data.csv'):
        df = pd.read_csv('raw_data.csv')

    else:
        df = pd.DataFrame([process_repo(repo) for repo in REPOS])
        df.to_csv('raw_data.csv')

    return df


# if __name__ == "__main__":
#     data = scrape_github_data()
#     json.dump(data, open("data.json", "w"), indent=1)

In [7]:
df = scrape_github_data()

In [9]:
df

,Unnamed: 0,repo,language,readme_contents
0,0,jtleek/datasharing,NaN,How to share data with a statistician\n=======...
1,1,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...
2,2,trimstray/the-book-of-secret-knowledge,NaN,"<p align=""center"">\n <a href=""https://github...."
3,3,ryanmcdermott/clean-code-javascript,JavaScript,# clean-code-javascript\n\n## Table of Content...
4,4,gothinkster/realworld,Shell,![RealWorld Example Applications](media/realwo...
5,5,vuejs/awesome-vue,NaN,"<p align=""center"">\n <br>\n <img width=""400""..."
6,6,tonsky/FiraCode,Clojure,## Fira Code: free monospaced font with progra...
7,7,hakimel/reveal.js,JavaScript,"<p align=""center"">\n <a href=""https://revealj..."
8,8,angular/angular.js,JavaScript,Using AngularJS with the Closure Compiler\n===...
9,9,reduxjs/redux,TypeScript,# <a href='https://redux.js.org'><img src='htt...
